# Segmenting & Clustering Neighborhoods in Toronto: Part 2

### Scrape Wikipedia page

In [ ]:
# use requests get text
import requests
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

# use BeautifulSoup to get the table
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url)
my_table = soup.find('table',{'class':'wikitable sortable'})

# read the table to a DataFrame
from IPython.display import display_html
import pandas as pd
df = pd.read_html(str(my_table))[0]

In [158]:
df = df.loc[df['Borough'] != 'Not assigned']

# if the neighbourhood is "Not assigned", assign the Borough name as the Neighbourhood
df.loc[df['Neighbourhood'] == 'Not assigned','Neighbourhood'] = df[df['Neighbourhood'] == 'Not assigned']['Borough']

df = df.assign(Latitude = "", Longitude = "")

### Geocode to get lattitude

In [178]:
#!conda install -c conda-forge geocoder --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
from ipywidgets import IntProgress
from IPython.display import display

n_neighborhoods = df.shape[0]

# progress bar
f = IntProgress(min=0, max=n_neighborhoods) # instantiate the bar
display(f) # display the bar

# geocode neighborhoods
geolocator = Nominatim(user_agent="my_agent")
for i in range(n_neighborhoods):
    row = df.iloc[i]
    address = (row.Neighbourhood + ", Toronto, ON")# + row.Postcode)
#    address = address.replace("Downtown","")

    try: 
        location = geolocator.geocode(address)
        df.loc[df['Neighbourhood']==row.Neighbourhood,'Latitude'] = location.latitude
        df.loc[df['Neighbourhood']==row.Neighbourhood,'Longitude'] = location.longitude
    except:
        print(address+" not found")

    f.value = i #progressbar

df.head()

IntProgress(value=0, max=211)

Humewood-Cedarvale, Toronto, ON not found
Caledonia-Fairbanks, Toronto, ON not found
CFB Toronto, Toronto, ON not found
Del Ray, Toronto, ON not found
Canada Post Gateway Processing Centre, Toronto, ON not found
Railway Lands, Toronto, ON not found
Humber Bay Shores, Toronto, ON not found
Beaumond Heights, Toronto, ON not found
Stn A PO Boxes 25 The Esplanade, Toronto, ON not found
Business Reply Mail Processing Centre 969 Eastern, Toronto, ON not found


,Postcode,Borough,Neighbourhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.7588,-79.3202
3,M4A,North York,Victoria Village,43.7327,-79.3112
4,M5A,Downtown Toronto,Harbourfront,43.6401,-79.3801
5,M5A,Downtown Toronto,Regent Park,43.6607,-79.3605
6,M6A,North York,Lawrence Heights,43.7228,-79.4509
